Fixed Costs and Linking of Variables.

Florida Home HealthCare is planning to planning to build a number of new facilities in Central Florida and has divided the map of the area into  regions. They want to locate the facilities such that all seven regions can be conviniently served by one facilty. Five possible sites are possible for constructing the new facilities. The region that can be served by the facilities in each of the sites are present by a number underneath the heading 'Possible Building sites' in the table below. Th number itself represents the cost of serving the customer from the given region by a professional stationed at the given facility. The table also included the estimiated demand of each region and for each potential facility, how much it would cost to build(1000s) and what its capacity would be. (Demands and Capacitied are estimated over an approximate planning horizon)

|           |                | Possible Building Sites |           |        |             |          |
|:---------:|:--------------:|:-----------------------:|:---------:|:------:|:-----------:|:--------:|
| Region iD | Region Demands |         Sanford         | Altamonte | Apopka | Casselberry | Maitland |
|     1     |       300      |           100           |           |   300  |             |          |
|     2     |       200      |           120           |    250    |        |     200     |    250   |
|     3     |       350      |                         |    350    |        |     150     |          |
|     4     |       450      |                         |           |   300  |             |    100   |
|     5     |       250      |           200           |    130    |        |             |          |
|     6     |       100      |                         |           |   180  |             |    110   |
|     7     |       150      |                         |           |        |     270     |    320   |
|           |  Building Cost |           450           |    650    |   550  |     500     |    525   |
|           |    Capacity    |           400           |    425    |   325  |     475     |    575   |
|           |                |                         |           |        |             |          |

In [2]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 59.1 MB/s eta 0:00:00


In [3]:
from pulp import *

Creating model instance

In [4]:
model = LpProblem('Facility Location Optimization', LpMinimize)

/usr/local/lib/python3.8/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Creating Variable for facility decision (Binary)

In [5]:
locations = ['Sanford', 'Altamonte', 'Apopka', 'Casselberry', 'MaitLand']
region = [1, 2, 3, 4, 5, 6, 7]

In [6]:
X= LpVariable.dicts("Facilty", locations, 0, 1, cat= "Integer")
X

{'Sanford': Facilty_Sanford,
 'Altamonte': Facilty_Altamonte,
 'Apopka': Facilty_Apopka,
 'Casselberry': Facilty_Casselberry,
 'MaitLand': Facilty_MaitLand}

Creating variable for allocation

In [7]:
allocation_keys = [(r, l) for r in region for l in locations]

In [8]:
Y = LpVariable.dicts("allocation", allocation_keys, 0, None, cat = 'Integer')
Y

{(1, 'Sanford'): allocation_(1,_'Sanford'),
 (1, 'Altamonte'): allocation_(1,_'Altamonte'),
 (1, 'Apopka'): allocation_(1,_'Apopka'),
 (1, 'Casselberry'): allocation_(1,_'Casselberry'),
 (1, 'MaitLand'): allocation_(1,_'MaitLand'),
 (2, 'Sanford'): allocation_(2,_'Sanford'),
 (2, 'Altamonte'): allocation_(2,_'Altamonte'),
 (2, 'Apopka'): allocation_(2,_'Apopka'),
 (2, 'Casselberry'): allocation_(2,_'Casselberry'),
 (2, 'MaitLand'): allocation_(2,_'MaitLand'),
 (3, 'Sanford'): allocation_(3,_'Sanford'),
 (3, 'Altamonte'): allocation_(3,_'Altamonte'),
 (3, 'Apopka'): allocation_(3,_'Apopka'),
 (3, 'Casselberry'): allocation_(3,_'Casselberry'),
 (3, 'MaitLand'): allocation_(3,_'MaitLand'),
 (4, 'Sanford'): allocation_(4,_'Sanford'),
 (4, 'Altamonte'): allocation_(4,_'Altamonte'),
 (4, 'Apopka'): allocation_(4,_'Apopka'),
 (4, 'Casselberry'): allocation_(4,_'Casselberry'),
 (4, 'MaitLand'): allocation_(4,_'MaitLand'),
 (5, 'Sanford'): allocation_(5,_'Sanford'),
 (5, 'Altamonte'): allocatio

Defining variables required for objective function.

In [9]:
allowed_facilities = [1, 0, 1, 0, 0,
                      1, 1, 0, 1, 1,
                      0, 1, 0, 1, 0,
                      0, 0, 1, 0, 1,
                      1, 1, 0, 0, 0,
                      0, 0, 1, 0, 1,
                      0, 0, 0, 1, 1]
allocation = dict(zip(allocation_keys, allowed_facilities))

In [10]:
cost_values =  [100, 0, 300, 0, 0,
                120, 250, 0, 200, 250,
                0, 350, 0, 150, 0,
                0, 0, 300, 0, 100,
                200, 130, 0, 0, 0,
                0, 0, 180, 0, 110,
                0, 0, 0, 270, 320]
cost = dict(zip(allocation_keys, cost_values))

In [11]:
facility_cost = [450000, 650000, 550000, 500000, 525000]
facility = dict(zip(locations, facility_cost))

OBJECTIVE

In [12]:
model+= lpSum(X[(l)]*facility[(l)] for l in locations) + lpSum(cost[(r, l)]*Y[(r, l)] for r in region for l in locations)

CONSTRAINTS

In [13]:
demands = [300, 200, 350, 450, 250, 100, 150]
capacity = [400, 425, 325, 475, 575]

In [14]:
facility_capacity = dict(zip(locations,  capacity))
facility_capacity

{'Sanford': 400,
 'Altamonte': 425,
 'Apopka': 325,
 'Casselberry': 475,
 'MaitLand': 575}

In [15]:
regional_demands = dict(zip(region, demands))
regional_demands

{1: 300, 2: 200, 3: 350, 4: 450, 5: 250, 6: 100, 7: 150}

In [16]:
for l in locations:
  model += lpSum((Y[(r, l)])*(allocation[(r, l)]) for r in region) <= facility_capacity[(l)] * X[l]

In [17]:
for r in region:
  model += lpSum((Y[(r, l)])*(allocation[(r, l)]) for l in locations) == regional_demands[(r)]

In [18]:
model.solve

<bound method LpProblem.solve of Facility_Location_Optimization:
MINIMIZE
650000*Facilty_Altamonte + 550000*Facilty_Apopka + 500000*Facilty_Casselberry + 525000*Facilty_MaitLand + 450000*Facilty_Sanford + 300*allocation_(1,_'Apopka') + 100*allocation_(1,_'Sanford') + 250*allocation_(2,_'Altamonte') + 200*allocation_(2,_'Casselberry') + 250*allocation_(2,_'MaitLand') + 120*allocation_(2,_'Sanford') + 350*allocation_(3,_'Altamonte') + 150*allocation_(3,_'Casselberry') + 300*allocation_(4,_'Apopka') + 100*allocation_(4,_'MaitLand') + 130*allocation_(5,_'Altamonte') + 200*allocation_(5,_'Sanford') + 180*allocation_(6,_'Apopka') + 110*allocation_(6,_'MaitLand') + 270*allocation_(7,_'Casselberry') + 320*allocation_(7,_'MaitLand') + 0
SUBJECT TO
_C1: - 400 Facilty_Sanford + allocation_(1,_'Sanford')
 + allocation_(2,_'Sanford') + allocation_(5,_'Sanford') <= 0

_C2: - 425 Facilty_Altamonte + allocation_(2,_'Altamonte')
 + allocation_(3,_'Altamonte') + allocation_(5,_'Altamonte') <= 0

_C3: - 

In [19]:
model.solve()

1

In [20]:
value(model.objective)

2374750.0

In [21]:
for i in X: 
    print('{} shipping {}'.format(i,X[i].varValue))

Sanford shipping 1.0
Altamonte shipping 1.0
Apopka shipping 0.0
Casselberry shipping 1.0
MaitLand shipping 1.0


In [22]:
for i in Y: 
    print('{} shipping {}'.format(i,Y[i].varValue))

(1, 'Sanford') shipping 300.0
(1, 'Altamonte') shipping None
(1, 'Apopka') shipping 0.0
(1, 'Casselberry') shipping None
(1, 'MaitLand') shipping None
(2, 'Sanford') shipping 100.0
(2, 'Altamonte') shipping 100.0
(2, 'Apopka') shipping None
(2, 'Casselberry') shipping 0.0
(2, 'MaitLand') shipping 0.0
(3, 'Sanford') shipping None
(3, 'Altamonte') shipping 0.0
(3, 'Apopka') shipping None
(3, 'Casselberry') shipping 350.0
(3, 'MaitLand') shipping None
(4, 'Sanford') shipping None
(4, 'Altamonte') shipping None
(4, 'Apopka') shipping 0.0
(4, 'Casselberry') shipping None
(4, 'MaitLand') shipping 450.0
(5, 'Sanford') shipping 0.0
(5, 'Altamonte') shipping 250.0
(5, 'Apopka') shipping None
(5, 'Casselberry') shipping None
(5, 'MaitLand') shipping None
(6, 'Sanford') shipping None
(6, 'Altamonte') shipping None
(6, 'Apopka') shipping 0.0
(6, 'Casselberry') shipping None
(6, 'MaitLand') shipping 100.0
(7, 'Sanford') shipping None
(7, 'Altamonte') shipping None
(7, 'Apopka') shipping None
(7, 'C